<a href="https://colab.research.google.com/github/gogela/Colab_COSMOS/blob/main/cosmos_nvidia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Colab NTB for NVIDIA cosmos

In [ ]:
#uninstal default pytorch & cuda
!pip uninstall -y torch torchvision torchaudio

!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
#cuda 12.6
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
!sudo dpkg -i cuda-keyring_1.1-1_all.deb
!sudo apt-get update
!sudo apt-get -y install cuda-toolkit-12-6

In [ ]:
!pip uninstall fastai -y
!pip install git+https://github.com/NVlabs/Pytorch_Retinaface.git@b843f45


In [ ]:
#new pytorch (night build)

#!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu126
!pip install torch==2.7.0.dev20250114+cu126 --index-url https://download.pytorch.org/whl/nightly/cu126
!pip install torchvision==0.22.0.dev20250114+cu126 --index-url https://download.pytorch.org/whl/nightly/cu126



In [ ]:
#!nvcc --version




import torch
print(torch.backends.cudnn.version()) #should be at least 90501


In [ ]:
#hf token
import os
from huggingface_hub import login
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
login(os.environ["HF_TOKEN"])



In [ ]:
# git
%cd /content/
# 1. Clone the repository and move to the directory (execute only once)
!(if [ ! -d "Cosmos" ]; then \
    git clone https://github.com/NVIDIA/Cosmos.git; \
  fi)
%cd Cosmos


In [ ]:
#!patch /content/Cosmos/cosmos1/models/diffusion/module/pretrained_vae.py
!cp /content/pretrained_vae.py /content/Cosmos/cosmos1/models/diffusion/module/pretrained_vae.py

In [ ]:
# upload sample video
!cp /content/sample_water.mp4 /content/sample.mp4

In [ ]:
%cd /content/Cosmos

# Not sure this is even needed
!apt-get update && apt-get install -y ffmpeg
!pip install --no-cache-dir -r requirements.txt
!pip install pynvml
#!pip install transformer_engine[pytorch]




In [ ]:
!pip uninstall transformer_engine
!pip cache purge
!pip install transformer_engine[pytorch]

Diffusion

In [ ]:
#download models - diffusion
#redirect uoutput to /dev/null - otherwise the lenghty output kills browser
!PYTHONPATH=$(pwd) python cosmos1/scripts/download_diffusion.py --model_sizes 7B --model_types Video2World >/tmp/download.log 2>&1
#!PYTHONPATH=$(pwd) python cosmos1/scripts/download_diffusion.py --model_sizes 7B 14B --model_types Text2World Video2World >/tmp/download.log 2>&1


In [ ]:
#Inference - diffusion
!(export PYTHONPATH=$(pwd) && \
  python cosmos1/models/diffusion/inference/video2world.py \
    --checkpoint_dir checkpoints \
    --diffusion_transformer_dir Cosmos-1.0-Diffusion-7B-Video2World \
    --prompt 'Pouring a glass of water in a kitchen' \
    --input_image_or_video_path /content/sample.mp4 \
    --num_input_frames 1 \
    --disable_prompt_upsampler \
    --video_save_folder video_output \
    --video_save_name Cosmos-1.0-Diffusion-7B-Video2World \
    --offload_diffusion_transformer \
    --offload_tokenizer \
    --offload_text_encoder_model \
    --offload_prompt_upsampler \
    --offload_guardrail_models)

Autoregression

In [ ]:
#Inference - autoregressive
%cd /content/Cosmos
!PYTHONPATH=$(pwd) python cosmos1/scripts/download_autoregressive.py --model_sizes 4B 5B >/tmp/download.log 2>&1

/content/Cosmos


In [ ]:
%cd /content/Cosmos
!CUDA_VISIBLE_DEVICES=0 PYTHONPATH=$(pwd) python cosmos1/models/autoregressive/inference/video2world.py \
    --input_type=text_and_video \
    --input_image_or_video_path=/content/sample.mp4  \
    --prompt="A video showing a kitchen sink and a tap. A hand holds a cup and water from the tap fills the cup" \
    --video_save_name=Cosmos-1.0-Autoregressive-5B-Video2World \
    --ar_model_dir=Cosmos-1.0-Autoregressive-5B-Video2World \
    --top_p=0.7 \
    --temperature=1.0 \
    --offload_guardrail_models \
    --offload_diffusion_decoder \
    --offload_ar_model \
    --offload_tokenizer \
    --offload_text_encoder_model